In [39]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier

In [2]:
df_1 = pd.read_csv("../data/Flight delays/flight_delays_train.csv")
df_2 = pd.read_csv("../data/Flight delays/flight_delays_test.csv")

In [3]:
df = pd.concat([df_1, df_2])

In [4]:
df_1.shape, df_2.shape, df.shape

((100000, 9), (100000, 8), (200000, 9))

In [5]:
df["month"] = df["Month"].str.replace("c-", "").astype(int)
df["WeekDay"] = df["DayOfWeek"].str.replace("c-", "").astype(int)
df["MonthDay"] = df["DayofMonth"].str.replace("c-", "").astype(int)

In [6]:
df["is_weekend"] = (df["WeekDay"] == 6) | (df["WeekDay"] == 7)

In [7]:
def categorize_dist(distance):
    if distance <= 1500:
        return "short-haul"
    elif 1500 < distance <= 3000:
        return "medium-haul"
    else:
        return "long_haul"

In [8]:
df["category"] = df["Distance"].apply(categorize_dist)

In [9]:
df["time"] = pd.to_datetime(df["DepTime"], format="%H%M", errors="coerce")

In [10]:
df['time'] = df["time"].fillna("00:00")

In [11]:
df["december_holidays"] = (df["month"] == 6) | (df["month"] ==7) | (df["month"] == 8)

In [12]:
df["time_of_day"] = df["time"].dt.hour

In [13]:
df.drop(["Month", "DayofMonth", "DayOfWeek", "DepTime", "time"], axis=1, inplace=True)

In [14]:
df.head()

,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min,month,WeekDay,MonthDay,is_weekend,category,december_holidays,time_of_day
0,AA,ATL,DFW,732,N,8,7,21,True,short-haul,True,19
1,US,PIT,MCO,834,N,4,3,20,False,short-haul,False,15
2,XE,RDU,CLE,416,N,9,5,2,False,short-haul,False,14
3,OO,DEN,MEM,872,N,11,6,25,True,short-haul,False,10
4,WN,MDW,OMA,423,Y,10,6,7,True,short-haul,False,18


In [23]:
cat_idx = np.where(df.dtypes == "object")

In [25]:
train_df = df[:100000]
test_df = df[100000:]

In [33]:
train_df["delayed"] = train_df["dep_delayed_15min"].map({"Y": 1, "N": 0})

C:\Users\Panache Branding\AppData\Local\Temp\ipykernel_8692\4232699012.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["delayed"] = train_df["dep_delayed_15min"].map({"Y": 1, "N": 0})


In [35]:
train_df.drop("dep_delayed_15min", axis=1, inplace=True)

C:\Users\Panache Branding\AppData\Local\Temp\ipykernel_8692\980255738.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.drop("dep_delayed_15min", axis=1, inplace=True)


In [38]:
X = train_df.drop("delayed", axis=1)
y = train_df["delayed"]

In [40]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3)

In [41]:
clf = CatBoostClassifier()

In [52]:
cat_idx = np.where(df.dtypes == "object")[0]

In [53]:
cat_idx

array([0, 1, 2, 4, 9], dtype=int64)

In [54]:
clf.fit(X_train, y_train, cat_features=[0, 1, 2, 4, 9], plot=True, verbose=False);

CatBoostError: Bad value for num_feature[non_default_doc_idx=0,feature_idx=8]="short-haul": Cannot convert 'b'short-haul'' to float